### Lab Assignment: Python Warmup and Logfile Analytics

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: August 20, 2023

---

This lab consists of two parts. Part 1 is the Python warmup. Part 2 is the logfile analytics in Spark.  
Answer the questions in this assignment, showing all code and solutions.

**Total points: 15**

---

### Part 1: Python Warmup

1) (1 PT) Rename this notebook to JupyterTutorial_[your_initials], where you will enter your initials in place of [your_initials].

2) (1 PT) In the cell below, enter a list of data science topics you find interesting.  Use the markdown style (you will need to change the style from the Code style).

##### List of Interesting Data Science Topics

1. Databases
2. APIs
3. Network Neuroscience

3) (1 PT) In the cell below, enter the following Python list:

some_vals = [-1, 6, 12, 34]  

You will use the Code style, run the cell, and print the list.

In [1]:
# define list
some_vals = [-1, 6, 12, 34]

# print list
print(some_vals)

[-1, 6, 12, 34]


4) (1 PT) Use a list comprehension to return a filtered list containing only the values greater than 6.  
Call this list *some_vals_filtered* and print it.

In [2]:
some_vals_filtered = [x for x in some_vals if x > 6]
print(some_vals_filtered)

[12, 34]


Next, a small pandas dataframe is constructed.

In [3]:
import pandas as pd

df = pd.DataFrame({'first_name': ['Andy','Crystal'],
                   'domain_facebook' : [1,1],
                   'domain_foursquare' : [0,0],
                   'age' : [20, 32]})
df

,first_name,domain_facebook,domain_foursquare,age
0,Andy,1,0,20
1,Crystal,1,0,32


5) (1 PT) In the cell below, write a list comprehension that returns the fields names in the dataframe `df` containing the string *domain*.  Run the cell to verify the correct result.

In [4]:
# list comprehension
domain_col_names = [x for x in df.columns if 'domain' in x]

# print the list to verify
print(domain_col_names)

['domain_facebook', 'domain_foursquare']


6) (1 PT) Use the list comprehension from (5) to index into `df` and show the data for columns containing *domain*

In [5]:
# print columns containing domain
df[domain_col_names]

,domain_facebook,domain_foursquare
0,1,0
1,1,0


7) (1 PT) In the cell below, print the *domain_facebook* column

In [6]:
# index and print domain_facebook col
print(df['domain_facebook'])

0    1
1    1
Name: domain_facebook, dtype: int64


8) (1 PT) In the cell below, print the row with index 1.

In [7]:
print(df.iloc[1])

first_name           Crystal
domain_facebook            1
domain_foursquare          0
age                       32
Name: 1, dtype: object


9) (1 PT) Next, you will cube the *age* column of `df` and assign the result to a new column called *agecube*.

Specifically, call the `apply` method with a `lambda function` inside to cube the *age* column.  
Print the dataframe.

In [8]:
# apply a lambda function to the age column
df['agecube'] = df['age'].apply(lambda x: x**2)
df

,first_name,domain_facebook,domain_foursquare,age,agecube
0,Andy,1,0,20,400
1,Crystal,1,0,32,1024


10) (1 PT) Given the list of strings below, form one string, placing semicolons between each word.  It should look like this:  

`'the;quick;brown;fox'`

Print the resulting string.

In [9]:
some_list = ['the','quick','brown','fox']

In [10]:
';'.join(some_list)

'the;quick;brown;fox'

### Part 2: Logfile Analytics

Import modules for Spark Session and regex 

In [11]:
from pyspark.sql import SparkSession
import re

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/06 15:00:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


11) (1 PT) Read in the logfile.txt data

In [12]:
# we can use the sc object to read the txt file
lines = sc.textFile('logfile.txt')

12) (1 PT) Count the number of rows of data

In [13]:
# can get the number of rows
lines.count()

360

13) (1 PT) Show the first 5 lines

In [14]:
# can use the take command
lines.take(5)

[' 01 ',
 '03/22 08:51:01 INFO   :.main: *************** RSVP Agent started ***************',
 ' 02 ',
 '03/22 08:51:01 INFO   :...locate_configFile: Specified configuration file: /u/user10/rsvpd1.conf',
 '03/22 08:51:01 INFO   :.main: Using log level 511']

14) (1 PT) Show all lines containing the word WARNING and write code to count them

In [15]:
# filter for lines with 'WARNING' in them
warning_lines = lines.filter(lambda x: 'WARNING' in x)

# print the warning lines by collecting them
print('Warning Lines', '\n', warning_lines.collect(), '\n')

# now want to count the number of lines so use count function
print( 'Warning Line Count', '\n', warning_lines.count() )

Warning Lines 
 ['03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.', '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.', '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.', '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.'] 

Warning Line Count 
 4


15) (1 PT) Write a word count program to count the number of each of these log levels:  

terms = 'WARNING|INFO|EVENT|PROTERR|TRACE'

In [16]:
terms = 'WARNING|INFO|EVENT|PROTERR|TRACE'

# split on pipe to have a list of the terms we have
terms = terms.split('|')

# using re library to split on two delimiters 
# warning and proterr have semicolons attatched which doesn't allow for native split command
word_count = lines.flatMap( lambda x: re.split(' |:', x) ) \
                .filter( lambda x: x in terms ) \
                .map( lambda x: (x, 1) ) \
                .reduceByKey( lambda x,y:x+y ) \
                .map( lambda x: (x[1], x[0]) ) \
                .sortByKey(False)

# print the outcome
word_count.take( len(terms) )

# note: Warning and Proterr have semi-colons after them which is causing an issue, to solve I used the re.split() command instead
# of the native list .split() command

[(145, 'INFO'), (119, 'TRACE'), (13, 'EVENT'), (4, 'WARNING'), (1, 'PROTERR')]